# Aim of the project

- the goal is to draw the three angels in the center of the face (nose)

- we first get the data from the data set called AFLW2000 it containes around 2000 photo

- the we build a supervised model to predict the pitch and roll and yaw

- then we test our model on photo and vedio

# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import warnings
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
warnings.filterwarnings('ignore')
from sklearn.svm import SVR
#from google.colab.patches import cv2_imshow
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


from sklearn.pipeline import Pipeline

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier,RandomForestClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import pickle
#import face_utils
import cv2
#import dlib
import math as m
import scipy.io as sio
%matplotlib inline

In [2]:
!pip install mediapipe

In [3]:
import mediapipe

# Showing The Data

In [4]:
df = pd.read_csv('data1.csv')
scaler = StandardScaler()
scaler.fit(df)
X = df.iloc[:,1:-3]
y = df.iloc[:,-3:]

In [5]:
X

,1,2,3,4,5,6,7,8,9,10,...,927,928,929,930,931,932,933,934,935,936
0,0.485651,0.686891,0.489194,0.639722,0.489803,0.647257,0.479730,0.562529,0.490558,0.621214,...,0.559712,0.485137,0.548437,0.492813,0.541429,0.500371,0.665669,0.471336,0.676091,0.464005
1,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,0.576966,...,0.522467,0.506956,0.511393,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387
2,0.317797,0.799190,0.321328,0.776498,0.318316,0.782066,0.311064,0.745973,0.321459,0.767816,...,0.334413,0.717794,0.330334,0.720868,0.328639,0.723342,0.373404,0.713071,0.377643,0.709798
3,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,0.637613,...,0.507567,0.500958,0.497731,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623
4,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,0.584701,...,0.520501,0.481912,0.510743,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,0.503578,0.685362,0.503318,0.626683,0.502649,0.641599,0.485319,0.553585,0.503746,0.606251,...,0.555657,0.487384,0.544162,0.494455,0.538516,0.500237,0.657289,0.475985,0.665278,0.469672
1848,0.515443,0.683951,0.532837,0.625785,0.518815,0.639885,0.511650,0.560010,0.535343,0.607497,...,0.549179,0.497122,0.541875,0.503230,0.540073,0.508401,0.629060,0.479723,0.635765,0.474628
1849,0.543907,0.689845,0.565286,0.635285,0.549288,0.646834,0.545584,0.562769,0.568493,0.616073,...,0.582101,0.493105,0.575992,0.500255,0.574974,0.506461,0.659800,0.479817,0.667754,0.470887
1850,0.586373,0.680103,0.604407,0.627700,0.593793,0.645848,0.602947,0.568153,0.609593,0.610428,...,0.663857,0.537300,0.654205,0.539478,0.648903,0.541426,0.734017,0.547668,0.743284,0.540139


In [6]:
y

,Yaw,Pitch,Roll
0,-0.399231,0.018227,0.085676
1,-0.175379,0.299208,-0.373374
2,-0.882169,1.198004,-1.033374
3,-0.026812,0.011965,-0.220662
4,0.223228,0.022474,-0.006809
...,...,...,...
1847,-0.367547,-0.429723,0.122791
1848,-0.015007,-0.485061,-0.313744
1849,-0.156035,0.567114,-0.108536
1850,-0.232617,-1.418751,0.175960


In [7]:

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.05,random_state=0)


In [8]:
print("X Train -->",X_train.shape)
print("Y Train -->",y_train.shape)
print("X Test  -->",X_test.shape)
print("Y Test  -->",y_test.shape)

X Train --> (1759, 936)
Y Train --> (1759, 3)
X Test  --> (93, 936)
Y Test  --> (93, 3)


# The Model

In [9]:
#using MultiOutputRegressor to get multi out(pitch, roll, yaw)

x_model = MultiOutputRegressor(SVR(kernel='rbf'))


x_model.fit(X_train , y_train)

predictions =  x_model.predict(X_train.values)

print(mean_squared_error(y_train.values, predictions))


print(r2_score(y_train.values, predictions))



test_predictions =  x_model.predict(X_test.values)



print(mean_squared_error(y_test, test_predictions))


print(r2_score(y_test, test_predictions))

0.45896389408150645
0.014673214299931034
0.21176311530628888
0.004528751280891881


In [10]:
# from sklearn.ensemble import VotingRegressor

# from sklearn.ensemble import GradientBoostingRegressor

# #build our pipeline

# M_P = make_pipeline(StandardScaler(),
#         MultiOutputRegressor(VotingRegressor(
#             estimators=[
#                 ('gb',GradientBoostingRegressor(n_estimators=10, random_state=0)),
#              ('lr', SVR()),
#              ]))
# )
# #fit the model
# M_P.fit(X_train.values, y_train.values)


# predictions =  M_P.predict(X_train.values)

# print(mean_squared_error(y_train.values, predictions))


# print(r2_score(y_train.values, predictions))



# test_predictions =  M_P.predict(X_test.values)



# print(mean_squared_error(y_test, test_predictions))


# print(r2_score(y_test, test_predictions))


# Draw Axis

In [11]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height/ 2

    x1 = size * (m.cos(yaw) *m.cos(roll)) + tdx
    y1 = size * (m.cos(pitch) * m.sin(roll) + m.cos(roll) * m.sin(pitch) *m.sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-m.cos(yaw) * m.sin(roll)) + tdx
    y2 = size * (m.cos(pitch) * m.cos(roll) - m.sin(pitch) * m.sin(yaw) * m.sin(roll)) + tdy
    
    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (m.sin(yaw)) + tdx
    y3 = size * (-m.cos(yaw) * m.sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

# Center around the nose

In [12]:
def getting_data_points(image):    
    #find nose
    Face_det = mediapipe.solutions.face_detection
    with Face_det.FaceDetection(min_detection_confidence = 0.6) as detector:
        results = detector.process(image)
        if results.detections != None:
            for face in results.detections:
                landmarks = face.location_data.relative_keypoints
                nose_pos = (int(landmarks[2].x * image.shape[1]), int(landmarks[2].y * image.shape[0]))





    faceModule = mediapipe.solutions.face_mesh
    points = []

    with faceModule.FaceMesh(static_image_mode=True) as faces:
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
        
          for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y
                    points.append(x)
                    points.append(y)
                    our_points = np.array([points])
    return(our_points, nose_pos)

# Using Photo

In [24]:

image = cv2.imread('Levi.webp')

landmarks, nose_pos = getting_data_points(image)
#use our model on the photo
predict = x_model.predict(landmarks)
pitch =  predict[0,0]
yaw = predict[0,1]
roll = predict[0,2]

lines_p = draw_axis(img=image, yaw = yaw, pitch= pitch, roll = roll, tdx=nose_pos[0], tdy=nose_pos[1])

cv2.namedWindow('finalImg', cv2.WINDOW_NORMAL)
cv2.imshow('lin',lines_p)
cv2.imwrite('PM.jpeg',lines_p)
cv2.waitKey(3000)
cv2.destroyAllWindows()

<img src= "PM.jpeg" alt ="Titanic" style='width: 800px;height:400px'>

# Vedio 

In [16]:

cap = cv2.VideoCapture(0)
#for resilutuion
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#40 frame by secod and width hight
out = cv2.VideoWriter('faiza1.mp4',fourcc, 40.0, (640,480))
print(cap.isOpened())
while(cap.isOpened()):
    #ret weaher its true so vedio will save in frame , false it will not
    ret, frame = cap.read()
    if ret == True:
        landmarks, nose_pos = getting_data_points(frame)
        predict = x_model.predict(landmarks)
        pitch =  predict[0,0]
        yaw = predict[0,1]
        roll = predict[0,2]
        frame = draw_axis(img=frame, yaw = yaw, pitch= pitch, roll = roll, tdx=nose_pos[0], tdy=nose_pos[1])
        print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out.write(frame)
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1)== ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()




True
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
640.0
480.0
